# データサイエンス基礎(with Titanic)

この問題は Kaggle https://www.kaggle.com/ のチュートリアル課題で、データサイエンスの手順を学習するためのサンプルでもあります。  
ここではそのデータを例に、実際のデータサイエンスの手順を学習してみます。

課題の Titanic は学習データと問題データが用意されており、学習データにだけ 「生存有無」 のフラグがあります。  
ここから機械学習による判定機か、判定のためのモデルを作成し、問題データ内の生死不明乗客が生き残ったかどうかを判定します。

データサイエンスの基礎の流れは以下の流れのようです。

1. データの概要確認
2. 各データの内容を確認
3. データ変換/データ補正/削除など
4. 学習モデルの作成
5. 判定データの作成

これらの流れを実際に行ってみます。

### 利用するライブラリの読み込み

In [ ]:
# データ解析や加工
import pandas as pd
import numpy as np
import random as rnd

# 可視化ツール
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## データの読み取り

まずは読み込まなければ始まりません。  
学習に使うモデルは `12.datascience/input/train.csv` で、問題ファイルは `12.datascience/input/test.csv` として用意されています。

まずはこれを読み込みましょう。

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

train_df.head()

各データの意味ですが

* PassengerId : 乗客ID
* Survived : 生き残ったかどうか
* Pclass : 部屋の等級
* Name : 氏名
* Sex : 性別
* Age : 年齢
* SibSp : タイタニックに同乗している兄弟/配偶者の数
* Parch : タイタニックに同乗している親/子供の数
* Ticket : チケット番号
* Fare : 料金
* Cabin : 客室番号
* Embarked : 乗った港

ということだそうです。  
どんなデータかわかったところで、データの状況を確認してみます。

## 各データの内容を確認

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

まずこの時点で Cabin の欠損量は冗談ではありません。  
これを補正なんて諦めた方がいいかも知れないので、潔く諦めてしまいましょう。

データの欠損がみられるのは Age / Fare / Cabin / Embarked だけですね。

次に考えるのは、これらを補正する価値があるのかどうかです。

### 年齢はどうだろうか？

年齢と生存率に何らかの有意性があるならば、補正する価値はありそうです。  
ということで、グラフを作成してみます。

In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

死亡した帯域と、生存した帯域で、20 台近辺はどちらも伸びてるので、単純に年齢層がそこに偏ってるだけだと考えられます。  
ただ、若年層の生存率は有意に高いですね。

これは補正してでも利用する価値はありそうです。

### Fare(料金)

In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Fare', bins=20)

形に有意性があるようには見えませんね。  
ですが生存率は価格の高い方が高そうか？

In [ ]:
temp_df = train_df[train_df['Fare'] < 20]
temp_df['Fare'] = temp_df['Fare'].apply(lambda x: 0 if x < 10 else 1)
temp_df.groupby(['Fare'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Fare を 金額 20 以下に絞り、 10 以下と10より高いエリアで区切って生存率を出してみると、４倍違います。  
Fare は生存率に関連するようです。

そうなれば、補正方法も考えてみます。  
金額的には、部屋のグレードと、乗った場所（航行距離）に影響を受けるはず。

In [ ]:
temp_df = train_df.copy()

def emverked_to_num(em):
    if em == 'S':
        return 0
    elif em == 'C':
        return 1
    elif em == 'Q':
        return 2
    else:
        return 1

temp_df['Embarked'] = temp_df['Embarked'].apply(emverked_to_num)
temp_df['Sex'] = temp_df['Sex'].map({ 'male':1, 'female':0 })

filtered_df = temp_df[temp_df['Pclass'] == 1]
plt.plot(filtered_df['Embarked'], filtered_df['Fare'], 'o')
plt.grid(True)

In [ ]:
filtered_df = temp_df[temp_df['Pclass'] == 2]
plt.plot(filtered_df['Embarked'], filtered_df['Fare'], 'o')
plt.grid(True)

In [ ]:
filtered_df = temp_df[temp_df['Pclass'] == 3]
plt.plot(filtered_df['Embarked'], filtered_df['Fare'], 'o')
plt.grid(True)

察するに距離によって最低金額は変わるらしい。  
等級は？

In [ ]:
filtered_df = temp_df[temp_df['Embarked'] == 0]
plt.plot(filtered_df['Pclass'], filtered_df['Fare'], 'o')
plt.grid(True)

In [ ]:
filtered_df = temp_df[temp_df['Embarked'] == 1]
plt.plot(filtered_df['Pclass'], filtered_df['Fare'], 'o')
plt.grid(True)

1等級とか値段が青天井か…  
ともあれ、相関性はありそう。

In [ ]:
filtered_df = temp_df[temp_df['Pclass'] == 1][temp_df['Embarked'] == 0]
plt.plot(filtered_df['Age'], filtered_df['Fare'], 'o')
plt.grid(True)

年齢ば参考にならなそうですね…  
ともあれ、補完方針は、同じ等級、同じ港の平均値としておきましょう。

### Cabin(客室番号)

In [ ]:
train_df['Cabin'].head(10)

…グラフにしなかったんじゃないです、できなかったんです。  
だって…ねぇ？えーって感じですよ。

グラフにせよ機械学習にせよ、単純なモデルで扱うには数字である必要があります。  
これ、どうしようもなくない？（汗

ということで、使いません（汗

### Embarked(乗船場所)

In [ ]:
train_df['Embarked'].describe()

UNIQUE 3 ということは、３種類の値しか存在していないということ。  
であるならば、数字に置き換えることができます。

In [ ]:
train_df['Embarked'].head(10)

In [ ]:
# S/C/Q しかないなら
def emverked_to_num(em):
    if em == 'S':
        return 0
    elif em == 'C':
        return 1
    elif em == 'Q':
        return 2
    else:
        return 3  # null のものが該当

temp_df = train_df.copy()
temp_df['EmbarkedNm'] = temp_df['Embarked'].apply(emverked_to_num)

g = sns.FacetGrid(temp_df, col='Survived')
g.map(plt.hist, 'EmbarkedNm')

これは傾向が別れましたね。  
乗船場所が 1:C, 2:Q の生存比率は、0:S の物より高そうです。

null のものを 3 として分離しましたが、運良く生存したものがわずかにいたようですね。  
であれば、無記名のものは 1:C と仮定して話を進めるのが良さそうです。

今度は欠損値を考えなくて良いデータも見てみましょう。

### PassengerId : 乗客ID

といってもさぁこれ

In [ ]:
train_df['PassengerId'].head()

ただの連番じゃん…こんなの生存に関わるかよ（＝＝；  
関わったとしても使い道が思い浮かばない…

### Pclass : 部屋の等級

In [ ]:
train_df['Pclass'].head()

数字ならそのまま使える。

In [ ]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Pclass')

有意差がありますね。


### Name : 氏名

Name ... はどう使っていいか迷いますね。  

In [ ]:
train_df['Name'].head()

定式化などはできませんが、手動でデータ補正をするとき、家族の推察には利用できそうですね。  
今回は諦めて捨ててしまいます。

### Sex : 性別

In [ ]:
train_df[["Sex", "Survived"]]\
    .groupby(['Sex'], as_index=False)\
    .mean()\
    .sort_values(by='Survived', ascending=False)

有意さとかそういうレベルではないです。  
グラフ作るまでもなくこの差…

### SibSp : タイタニックに同乗している兄弟/配偶者の数

In [ ]:
train_df[["SibSp", "Survived"]]\
    .groupby(['SibSp'], as_index=False)\
    .mean()\
    .sort_values(by='Survived', ascending=False)

これもグラフにするまでもなく有意差が出ますね。

### Parch : タイタニックに同乗している親/子供の数

* Ticket : チケット番号
* Cabin : 客室番号
* Embarked : 乗った港

In [ ]:
train_df[["Parch", "Survived"]]\
    .groupby(['Parch'], as_index=False)\
    .mean()\
    .sort_values(by='Survived', ascending=False)

これも差が出ますね…親が我が子を庇うとかそういう話でしょうか…。  
親の身は有限だからか、兄弟の数が増えると生存率が下がっていきます。

### Ticket : チケット番号

これも扱いに困りますね

In [ ]:
train_df['Ticket'].head()

あーうん、無理！  
これ扱うの無理だわ！

という事で捨て捨て！

## データ変換/データ補正/削除など

さて、前述の結果から、消す物残す物、利用する物を順次決めていきます。  
まず、消すといった物を消していきましょう。

対象は

* PassengerId : 乗客ID
* Name : 氏名
* Ticket : チケット番号
* Cabin : 客室番号

あとは途中で作ってしまった `EmbarkedNm` も削除しておきましょうか。

In [ ]:
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
# PassengerId は回答用インデックスなので、削除しない
test_df = test_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

train_df.head()

次に欠損値の補完を行いましょう。

最初の対象は Fare 値です。  
先ほど記述したように、同じ等級/港の平均値とします。

In [ ]:
# Pclass と Embarked でグループ化、Fare の平均値を計算、
# 計算結果を fillna で空欄に挿入します。
test_df['Fare'] = test_df.groupby(['Pclass', 'Embarked'])['Fare']\
        .apply(lambda d: d.fillna(d.mean()))
test_df.info()

In [ ]:
# 連続値はアルゴリズムによっては扱いづらいので、10 ごとの整数値に直してしまいます。
train_df['Fare'] = train_df['Fare'].apply(lambda v: int(v / 10))
test_df['Fare'] = test_df['Fare'].apply(lambda v: int(v / 10))
test_df.head()

次の対象は「Age(年齢)」です。

方針としては3つ方策があります。

1. データを徹底的に眺めて、正しそうな値をマニュアルで補完する
2. とりあえず平均値の様な値で埋めてしまう
3. 欠損値を欠損を示す値に設定して処理を進めてしまう

ただし、３ の方法は、データの属性がラベル的な物である事が前提となります。  
はて、これは少しいい方向にながれそうです。

### データのクラス化

年齢に関して見ると、かなりバラバラの数値です。  
そのまま学習したところで、同じ年齢でなければ正しい判定が出てこない可能性があります。

そこで、一定の年齢範囲の単位でグループ化してしまいます。

前述のグラフを見る限り

* 5 才以下
* 6 - 15才
* 16 - 30才
* 31-60才
* それ以上

でクラス分類した方が良さそうな感じですね。  
このついでといってしまうとあれですが、欠損は別のラベルを振ってしまいます。

In [ ]:
def class_with_age(age):
    if age < 5:
        return 0
    elif age < 16:
        return 1
    elif age < 30:
        return 2
    elif age < 60:
        return 3
    elif age != None:
        return 4
    else:
        return 5

train_df['Age'] = train_df['Age'].apply(class_with_age)
test_df['Age'] = test_df['Age'].apply(class_with_age)
train_df.head()

### 値の数字化

次に、数字でない物を数字に変換しましょう。  
これは機械学習のモデルが文字列をどう扱っていいか不明になるためです。

In [ ]:
def emverked_to_num(em):
    if em == 'S':
        return 0
    elif em == 'C':
        return 1
    elif em == 'Q':
        return 2
    else:
        return 1  # null のものが該当(test 側は null ではない)

train_df['EmbarkedNm'] = train_df['Embarked'].apply(emverked_to_num)
test_df['EmbarkedNm'] = test_df['Embarked'].apply(emverked_to_num)

train_df = train_df.drop(['Embarked'], axis=1)
test_df = test_df.drop(['Embarked'], axis=1)

train_df.head()

In [ ]:
train_df['Sex'] = train_df['Sex'].map({'female':0, 'male':1})
test_df['Sex'] = test_df['Sex'].map({'female':0, 'male':1})
train_df.head()

### 新しい属性のデータ列を作成する

複数のデータ列を組み合わせた列を作成することで、何らかの有意な結果が得られる場合があります。

In [ ]:
combine = [train_df, test_df]

# 家族サイズ（自分含む）
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']]\
    .groupby(['FamilySize'], as_index=False)\
    .mean()\
    .sort_values(by='Survived', ascending=False)

In [ ]:
# 独り身かどうか
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']]\
    .groupby(['IsAlone'], as_index=False)\
    .mean()\
    .sort_values(by='Survived', ascending=False)

## 学習モデルの作成

本当ならここでこの学習データに対して複数の学習モデルを適用し、数値の良い物を利用するのですが、今回は面倒になってしまったので、RandomForest でお茶を濁します（苦笑）。

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()

X_train.shape, Y_train.shape, X_test.shape

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

90% 出ましたね…もう少し上がればいいのですが（汗

## 問題を解いてみる

という事で、こうしてできた学習機に実際に回答させてみます。

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('./submission.csv', index=False)

そしてこれを Kaggle にアップロードします。

ちなみに、Kaggle 上に乗っているチュートリアル、`Titanic Data Science Solutions` の方が精度はいいです（苦笑